# Get StrataScratch Subscribers with .subscribers() Youtube API

In [ ]:
!pip install google-auth
!pip install google-auth-oauthlib

In [2]:
import pandas as pd
import os
import pickle
import time
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import googleapiclient.discovery
import googleapiclient.errors

load credentials

In [3]:
credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading Credentials From File...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

# Google's Request
from google.auth.transport.requests import Request

# If there are no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching New Tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly'
            ]
        )

        flow.run_local_server(port=8080, prompt='consent')#,authorization_prompt_message='')
        credentials = flow.credentials

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as f:
            print('Saving Credentials for Future Use...')
            pickle.dump(credentials, f)

Loading Credentials From File...
Refreshing Access Token...


In [4]:
#Get first response for sub/comms subscriptions that are allowed:
def get_subcomms_subscriptions_first_response(channelId):
        api_service_name = "youtube"
        api_version = "v3"

        youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)
        request = youtube.subscriptions().list(part="snippet",channelId=channelId)
        # request = youtube.subscriptions().list(part="snippet,contentDetails",pageToken='CLYHEAE', maxResults=maxResults,mySubscribers=True)
        response = request.execute()
        return response, request, youtube

In [5]:
def get_subscriber_ids(df,response):
    for sub in response['items']:
        if sub['kind'] == 'youtube#subscription': 
            
            response_kind =sub['kind']
            publishedAt=sub['snippet']['publishedAt']
            channel_title =sub['snippet']['title']
            channel_description =sub['snippet']['description']
            channelId = sub['snippet']['resourceId']['channelId']
            sub_channelId = sub['snippet']['channelId']        

            try:
                if response['nextPageToken'] != None:
                    pageToken = response['nextPageToken']
                    df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channel_description':channel_description,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':pageToken},ignore_index=True)    
            except:
                #save data in pandas df
                df = df.append({'response_kind': response_kind, 'publishedAt':publishedAt, 'channel_title':channel_title,'channelId':channelId,'sub_channelId':sub_channelId,'pageToken':'none'},ignore_index=True)
    
    return df

In [6]:
def get_subs_pagination(df,request,response,youtube):
    while 1:
        try:
            request = youtube.subscriptions().list_next(previous_request=request, previous_response=response)
            response = request.execute()
            df = get_subscriber_ids(df,response)
        
        except:
            break
            
    
    return df,request,response

In [71]:
def main_getter(df,chan_ids,start):  #this uses all other functions to get all subscriptions
    for channelId in chan_ids[start:goal]:

        try:        
            response ,request, youtube = get_subcomms_subscriptions_first_response(channelId)
            #turn first response to df
            df = get_subscriber_ids(df,response)
            print('df length: ' + str(len(df)))

            df,request,response = get_subs_pagination(df=df,response=response ,request=request, youtube=youtube)
            print("user count: " +str(df.sub_channelId.nunique()))
        except:
            pass
        
    last_user = df.sub_channelId.nunique() + start #create last user variable to use for name
    return df, last_user

In [61]:
def main_getter(df,dfall,chan_ids,start):  #this uses all other functions to get all subscriptions
    duplicates = 0
    completed = dfall.sub_channelId.unique() 
    for channelId in chan_ids[start:goal]:
            
        if channelId in completed:
            duplicates =duplicates + 1
            if duplicates % 15 ==1:
                print('avoided duplicate')
            
            pass
        else:


            try:        
                response ,request, youtube = get_subcomms_subscriptions_first_response(channelId)
                #turn first response to df
                df = get_subscriber_ids(df,response)
                if len(df) % 15 == 1:
                    print('df length: ' + str(len(df)))

                df,request,response = get_subs_pagination(df=df,response=response ,request=request, youtube=youtube)
                if len(df.sub_channelId.nunique()) % 15 == 1:
                    print("user count: " +str(df.sub_channelId.nunique()))
            except:
                pass
    
    return df, duplicates

In [30]:
start

631

In [46]:
start

695

## Main

In [9]:
# start=631
# print(start)

631


In [44]:
goal

1695

In [72]:
# create chan_ids from imported df for all youtube scraping:
dfo = pd.read_csv('all_sqlbelle_youtube_comments_4200rows.csv',low_memory=False)
chan_ids = dfo.authorChannelId.unique()
chan_ids = chan_ids[1:]

In [90]:
#save start as just following the last unique sub_channelId
last_unique_scId = df.loc[df.index[-1],'sub_channelId']
last_Id_idx = list(chan_ids).index(last_unique_scId)
start = last_Id_idx +1;start

1126

In [91]:
#main

df = pd.DataFrame()
# goal = start +300

goal = start + 600

# dfo = pd.read_csv('strata_unique_commenters_with_1k_of_subscribers.csv',low_memory=False)
# df_combo = pd.read_csv('all_ss_comsubs_combined.csv',low_memory=False)


#########Grab everything using this function
df, last_user = main_getter(df,chan_ids=chan_ids,start=start)


# df.to_csv(f'all_ss_comsubs_subscriptions_{start}_{last_user}.csv',index=False) # save as csv
# print(f'Saved: all_ss_comsubs_subscriptions_{start}_{last_user}.csv') 
df.to_csv(f'competitor_sqlbelle_commenters_subscriptions_{start}_{last_user}.csv',index=False) # save as csv
print(f'Saved: competitor_sqlbelle_commenters_subscriptions_{start}_{last_user}.csv') 

# dfall = pd.read_csv('all_ss_comsubs_combined.csv',low_memory=False) #load combined csv
# dfall = pd.concat([dfall,df]) # concat to new df

# dfall = df
dfall = pd.read_csv('all_sqlbelle_coms_combined.csv',low_memory=False) #load combined csv
dfall = pd.concat([dfall,df]) # concat to new df
dfall = dfall.drop_duplicates() #drop allo= duplicates

# save newly combined and print the number of unique users
# dfall.to_csv('all_ss_comsubs_combined.csv',index=False) ;print(f'total combined Uniques: '+ str(dfall.sub_channelId.nunique())) 
dfall.to_csv('all_sqlbelle_coms_combined.csv',index=False) ;print('total combined Uniques: '+ str(dfall.sub_channelId.nunique() )+' / '+ str(dfo.authorChannelId.nunique())) 


# start=last_user #set up start for next time around
last_unique_scId = df.loc[df.index[-1],'sub_channelId']
last_Id_idx = list(chan_ids).index(last_unique_scId)
start = last_Id_idx +1

time.sleep(120)

print(f'start = {start}'+' / '+ str(dfo.authorChannelId.nunique()))
df.tail(3)

AttributeError: 'DataFrame' object has no attribute 'sub_channelId'